###### Needed installation

In [1]:
pip install flatten_json

Note: you may need to restart the kernel to use updated packages.


##### Main libraries

In [90]:
#basic libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#scrapping libraries

import requests, json
from flatten_json import flatten # to simplify nested json files in pandas df

In [91]:
pd.set_option('max_colwidth', None)

#### ------------------------------------------------------------------------------------------------------

#### Because the season 2020-21 is over there isn't a next round so I will use the last one and I will update the rest of the code when the season 2021-22 begins.

In [92]:
#Retrieving last season

df_odds = pd.read_csv('http://www.football-data.co.uk/mmz4281/2021/SP1.csv', header = 0, usecols = ['HomeTeam', 'AwayTeam','B365H', 'B365D', 'B365A'])
df_odds = df_odds.tail(10)

df_odds.to_excel('coming_round.xlsx')

In [93]:
df_odds

,HomeTeam,AwayTeam,B365H,B365D,B365A
370,Levante,Cadiz,1.75,4.10,4.10
371,Celta,Betis,3.20,3.80,2.10
372,Eibar,Barcelona,4.33,4.75,1.65
373,Elche,Ath Bilbao,1.95,3.60,3.80
374,Huesca,Valencia,1.65,4.20,5.00
375,Osasuna,Sociedad,5.00,4.33,1.61
376,Real Madrid,Villarreal,1.40,5.00,6.50
377,Valladolid,Ath Madrid,9.00,5.25,1.33
378,Granada,Getafe,2.62,3.40,2.60
379,Sevilla,Alaves,1.50,4.33,6.50


#### ------------------------------------------------------------------------------------------------------

#### The following cells will be replaced and updated as soon as the data for the next season is available. This part of the code was intended to retrieve the coming round so it doesn´t work anymore. It will be displayed for academical reasons as ´markdown'.

##### 1) Scrapping upcoming events

# upcoming events

url = "https://betsapi2.p.rapidapi.com/v1/bet365/upcoming"

querystring = {"sport_id":"1","league_id":"10041110","page":"1"}

headers = {
    'x-rapidapi-key': "aea4b96da0mshe2e45935146cf96p11c6a9jsnc73ba2aae11c",
    'x-rapidapi-host': "betsapi2.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

###### 1.1) Upcoming events : from json to dataframe

upcoming_events=json.loads(response.text[67:-1])

response_columns = ['id','sport_id','time','time_status','league_id','league_name','home_id','home_name','away_id','away_name','ss','our_event_id','r_id','updated_at']
df_upc_events = pd.DataFrame(columns=response_columns)

for match in upcoming_events:
    match_list = flatten(match)
    df_upc_events = df_upc_events.append(match_list, ignore_index=True)

df_upc_events = df_upc_events.drop(['away_id', 'home_id','league_id','league_name','r_id','ss','time', 'time_status','sport_id','our_event_id','updated_at'], axis = 1)
df_upc_events = df_upc_events[0:10]

df_upc_events

##### 2) Scrapping upcoming events odds

url = "https://betsapi2.p.rapidapi.com/v3/bet365/prematch"

headers = {
    'x-rapidapi-key': "aea4b96da0mshe2e45935146cf96p11c6a9jsnc73ba2aae11c",
    'x-rapidapi-host': "betsapi2.p.rapidapi.com"
    }

df_upc_odds = pd.DataFrame(columns = ['id','1','X','2'])

for i in df_upc_events.id:

    querystring = {"FI":i}
    
    upcoming_odds = requests.request("GET", url, headers=headers, params=querystring)
    
    # turn output into a daframe
    
    upcoming_odds =json.loads(upcoming_odds.text[24:-2])                                    # Fitting the output
    upcoming_odds = flatten(upcoming_odds)

    FI = upcoming_odds.get('FI')
    result_1 = upcoming_odds.get('main_sp_full_time_result_odds_0_odds')                    
    result_X = upcoming_odds.get('main_sp_full_time_result_odds_1_odds')
    result_2 = upcoming_odds.get('main_sp_full_time_result_odds_2_odds')                    # Retrieving key data
    
    list = [FI,result_1,result_X, result_2]       
    a_series = pd.Series(list, index = df_upc_odds.columns)                                 # Gathering key data 
    df_upc_odds = df_upc_odds.append(a_series, ignore_index=True)

upcoming_odds

##### 3) Merging both datasets

df_odds = df_upc_events.merge(df_upc_odds, how = 'left', on = 'id')

# To match names in the historical dataset

renaming_teams = {'CD Alaves':'Alaves','Athletic Bilbao':'Ath Bilbao','Real Betis':'Betis','Atletico Madrid':'Ath Madrid','Celta Vigo':'Celta','Real Sociedad':'Sociedad'}

df_odds['home_name'].replace(renaming_teams, inplace = True)
df_odds['away_name'].replace(renaming_teams, inplace = True)

df_odds.to_excel('coming_round.xlsx')